# Assignment 2b Notebook: Automated Design of AI Agents with Genetic Programming
In this assignment, you will further iterate on to your Assignment 2a implementation of random parse tree generation to realize a full genetic programming (GP) implementation. Like Assignment Series 1, you will leverage components you've already implemented in this assignment. From your previous assignment implementations, copy over the following files:
* base_evolution.py
* selection.py
* tree_genotype.py

As usual, be careful not to overwrite any of the provided files that may have been modified since previous assignments. To begin the assignment, execute the following cell. **If you implemented your genotype in a new file, be sure to import it in the next cell!**

In [ ]:
# Configure this notebook to automatically reload modules as they're modified
# https://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from snake_eyes import read_config
from fitness import play_GPac
from selection import *
from tree_genotype import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

print('The first cell has been executed!')

## Recombination
Assuming you've correctly implemented population initialization in Assignment 2a, we'll begin this assignment by implementing recombination with subtree crossover. Like Assignment Series 1, your recombination method should take a mate as input and recombine the genes of `self` and `mate`, assign the recombination to the `gene` member variable of `child` and then return child. Recall that you should somehow enforce a configurable maximum tree depth throughout this assignment and this applies to children produced with recombination and mutation.

How you implement subtree crossover in practice depends on your implementation of the parse tree gene and is thus open-ended. Implement `TreeGenotype.recombine` and test your implementation by executing the following cell.

In [ ]:
# read config
config = read_config('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# initialize population
random_population = TreeGenotype.initialization(25, **config['initialization_kwargs'])

# perform recombination
children = list()
for idx in range(len(random_population)):
    child = random_population[idx].recombine(random_population[idx+1%len(random_population)],
                                             **config['recombination_kwargs'])
    children.append(child)

# print recombined trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}r.txt','w') as f:
        f.write(individual.print())
del random_population, children, config

# evaluate tree files
!python tree_check.py treeTests/tree*r.txt

## Mutation
Recall that mutation in GP is mutually exclusive with recombination. That is to say that mutation in GP is used to directly produce children by mutating a copy of the parent.

Using your parse tree gene implementation, implement subtree mutation in `TreeGenotype.mutate` and test your implementation by executing the next cell. Don't forget to enforce max tree depth like in recombination!

In [ ]:
# read config
config = read_config('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# initialize population
random_population = TreeGenotype.initialization(25, **config['initialization_kwargs'])

# perform mutation
children = list()
for individual in random_population:
    children.append(individual.mutate(**config['mutation_kwargs']))

# print mutated trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}m.txt','w') as f:
        f.write(individual.print())
del random_population, children, config

# evaluate tree files
!python treeCheck.py treeTests/tree*m.txt

## Implementing Genetic Programming
By this point, you should have a complete implementation of your genotype and we can turn our attention to implementing the complete GP algorithm.

### Evaluation and Parsimony Pressure
Recall from the lecture and videos by Dr. Koza that GP tends to produce individual genotypes of increasing size without a mechanism to curb this. You will implement a parsimony penalty to encourage your GP to produce more compact trees (in addition to the depth limits you already have). With this mechanism of parsimony pressure, you will penalize the fitness of a solution based on their size. $fitness(i)=raw\_fitness(i)-C_p*size(i)$ where $C_p$ is a parsimony coefficient from your config file and $size(i)$ is a function that returns the size of individual $i$. The two most obvious metrics of tree size are tree depth and node count, but you are encouraged with experiments to use the size metric that performs best.

**Note**: The implementation of a parsimony penalty is nearly identical to the penalty-based constraint satisfaction method you implemented in Assignment 1c. Like Assignment 1c, *it is not meaningful to compare penalized fitness with unpenalized fitness* and we require that you use penalized fitness for evolution and raw fitness for analysis.

In the file `gpac_population_evaluation.py`, implement the `basic_population_evaluation` function that performs fitness evaluations on an input population using the input configuration parameters and assigns parsimony-penalized fitness, raw fitness, and saves the log of the fitness evaluation. Now test your implementation by executing the following cell.

In [ ]:
from gpac_population_evaluation import basic_population_evaluation

config = read_config('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

example_population = TreeGenotype.initialization(25, **config['initialization_kwargs'])
for individual in example_population:
    individual.raw_fitness = None
    individual.log = None

# calling your function to test things out (this function is called the same as in notebook 1c)
basic_population_evaluation(example_population, **config['fitness_kwargs'])

missing = len([True for individual in example_population if individual.fitness is None])
print(f'Individuals with unassigned (penalized) fitness: {missing}')
raw_missing = len([True for individual in example_population if individual.raw_fitness is None])
print(f'Individuals with unassigned raw fitness: {raw_missing}')
print(f'Number of complete fitness evaluations performed: {len(example_population - max(missing, raw_missing))}')
if missing == 0 and raw_missing == 0:
    fitnesses = [individual.fitness for individual in example_population]
    print(f'Average fitness of population: {statistics.mean(fitnesses)}')
    print(f'Best fitness in population: {max(fitnesses)}')
    raw_fitnesses = [individual.raw_fitness for individual in example_population]
    print(f'Average raw fitness of population: {statistics.mean(raw_fitnesses)}')
    print(f'Best unpenalized (raw) fitness in population: {max(raw_fitnesses)}')
    best_log = max(example_population, key=lambda ind: ind.fitness).log

    print(f'Found log of highest scoring individual? {bestLog is not None}')
    with open(game_log_path, 'w') as f:
        [f.write(f'{line}\n') for line in bestLog]

    print(f"The log of the most fit individual was written to {game_log_path}")
else:
    print('Ensure the entire population is evaluated then execute this cell again')

del example_population, config

### Child Generation
With fitness evaluation implemented, we can now perform parent selection and move to implementing child generation with the `GeneticProgrammingPopulation` class. This class inherits the `BaseEvolutionPopulation` class from last assignment and should be able to directly use the inherited initialization and survival selection methods without modification. Child generation is different in GP, as mentioned previously in this notebook, so you need to implement the GP version of `generate_children` in the `GeneticProgrammingPopulation` class. Once complete, test your implementation in the following cell.

In [ ]:
config = read_config('./configs/green2b_config.txt', globalVars=globals(), localVars=locals())

# full initialization of your GP population
example_EA = GeneticProgrammingPopulation(**config['EA_configs'], **config)
evaluate_population(example_EA.population, **config['fitness_kwargs'])
example_EA.evaluations = len(example_EA.population)
fitnesses = [individual.fitness for individual in example_EA.population]
print(f'Average fitness of population: {statistics.mean(fitnesses)}')
print(f'Best fitness in population: {max(fitnesses)}')
print(f'Number of fitness evaluations: {example_EA.evaluations}')

# generate children
children = example_EA.generate_children()
evaluate_population(children, config['fitness_kwargs'])
example_EA.evaluations += len(children)
children_fitnesses = [individual.fitness for individual in children]
print(f'Average fitness of children: {statistics.mean(children_fitnesses)}')
print(f'Best fitness of children: {max(children_fitnesses)}')
print(f'Number of fitness evaluations: {exampleEA.evaluations}')

# print children trees to files
for idx, individual in enumerate(children):
    with open(f'treeTests/tree{idx}c.txt','w') as f:
        f.write(individual.print())
del example_EA, children, config, fitnesses, children_fitnesses

# evaluate tree files
!python tree_check.py treeTests/tree*c.txt

## Single Run Experiment
At this point, you should have implemented the full GP algorithm to evolve GPac controllers! Now put all the components together and implement a single-run experiment with 2,000 fitness evaluations in the next cells.

In [ ]:
from math import inf

# implement your GP here
def genetic_programming_search(number_evaluations, config_filename):
    best_fitness = -inf
    data = None
    
    # Parse the config and implement your EA here.
    # Feel free to focus on implementation first and then return for data collection.
    
    return best_fitness, data

In [ ]:
# testing your implementatino out
print(genetic_programming_search(2000, './configs/green2b_config.txt'))

## Tuning
As is the case with all evolutionary algorithms, tuning your genetic programming algorithm is very important! In this case, it's also a graded component of the Algorithmic section of this assignment. As was the case with Assignment 1b, hand-tuning the algorithm parameters using a couple runs at a time is good enough for this class. Like Assignment 1b, you are expected to tune your algorithm parameters to a degree that your full GP outperforms the ramped half-and-half algorithm used in Assignment 2a (note that the default parameters are intentionally bad). While tuning, it can be helpful to ask yourself the following:
 * How many generations of evolution will occur based on $\mu$ and $\lambda$?
 * Is premature convergence occuring (e.g., max fitness is mediocre and does not increase)?
 * Is a loss of diversity preventing improvement (e.g., mean and best fitness approach convergence prematurely)?
 * If tuning $k$, is $k$-tournament actually being used as a selection method?
 * Have the problem instance or search space parameters been manipulated in a way that invalidates a comparison with ramped half-and-half (i.e., avoid manipulating the game parameters and tree depth limits)?
 * Is the parsimony coefficient tuned well? (Note: you'll need to consider this explicitly if attempting the YELLOW deliverable)

Use the following cell to tune the parameters in './configs/green2b_config.txt'.

In [ ]:
# feel free to change these values and re-run this cell as much as you'd like
number_runs = 5
number_evaluations = 2000
config_filename = './configs/green2b_config.txt'

# tuning runs can be called here


## Multi-Run Experiments
Implement a full 30-run experiment with 2,000 fitness evaluations per run. For each generation, log the average fitness and best fitness of the current population. Average this data across all 30 runs to produce a plot of fitness vs evaluations with average and best fitness averaged across all 30 runs. For each run, log the best (unpenalized/raw) fitness encountered for statistical analysis with your results from Assignment 2a. For the individual with the highest fitness of the full 30-run experiment, save the log for visualization and informal analysis and comparison with the best agent from Assignment 2a.

In [ ]:
number_runs = 30
number_evaluations = 2000
config_filename = './configs/green2b_config.txt'

# Implement your 30-run experiment here


## Report
See Assignment Description Document for report details.

# YELLOW Deliverable
Now that you've completed the GREEN deliverable and conducted an experiment with one of the parsimony metrics (e.g., tree depth or tree size), experiment with the other metric and tune the parsimony coefficient. You are welcome to re-use the other parameters you used in your GREEN configuration, but some attempt at tuning the parsimony coefficient to account for the new parsimony metric is required.

***Note***: You should add a corresponding parameter to your config files that enables the selection of a parsimony metric.

Use the following 2 cells to tune and conduct a 30-run experiment, respectively:

In [ ]:
# feel free to change these values and re-run this cell as much as you'd like
number_runs = 5
number_evaluations = 2000
config_filename = './configs/yellow2b_config.txt'

# tuning runs can be called here


In [ ]:
number_runs = 30
number_evaluations = 2000
config_filename = './configs/yellow2b_config.txt'

# Implement your 30-run experiment here


# RED Deliverables (Bonus)
For each RED deliverable you attempt, create new notebook cells as necessary and do not overwrite or break compatibility with your previous experimentation. Bonus RED deliverables are typically somewhat open-ended, but this is particularly true in Assignment Series 2. As such, you may encounter unanticipated design decisions and obstacles that you should figure out how to overcome. TA assistance is available to answer questions, though you are expected to experiment and investigate solutions/answers prior to seeking this assistance. You are welcome to re-use tuned parameters between these experiments instead of more rigorous tuning, though you may observe more interesting results with per-deliverable tuning. Finally, the components implemented for the Assignment 2b RED deliverables may reappear in RED deliverables of Assignment 2c. The RED deliverables for Assignment 2c are undecided, but recall that Assignment 2c is worth double points and percentage-based bonuses are consequently quite valuable to the overall Assignment Series 2 grade.

***Note***: When attempting deliverables RED 3 and 4, feel free to use the provided random Pac-Man controller as the static agent.

Create cells below to perform your RED deliverable experiments.

# Side Note: Canonical Genetic Programming

Should you apply GP after this class, you should know that the GP algorithm taught in this class (as described in the course textbook) differs somewhat from the algorithm canonically used in GP. Notably, the textbook has certain important omissions regarding the Ramped Half-and-half algorithm and the GP evolutionary cycle. In practice, the Ramped Half-and-half algorithm uses a `grow` method which ensures at least 1 branch reaches the depth limit. This can be difficult to implement, and has little impact on this assignment, so we don't require the implementation of the canonical version of the algorithm.

More importantly, however, is that the canonical GP evolutionary cycle is generational in nature. In the canonical Generational GP algorithm, $\mu$ children are created each generation via recombination, mutation, or *reproduction* and the children directly replace the parents without survival selection. Notably, the reproduction operation involves directly copying the selected parent to the children pool and this enables behavior similar to steady state survival. The algorithm you implement for this assignment series is much more similar to a Genetic Algorithm in nature and you should be aware of this distinction if you continue to work with GP. For more information see [here](https://geneticprogramming.com/about-gp/gp-workflow/).